In [1]:
from src.actors.StudentInfo import StudentInfo
from src.certificate_authority.CertificateAuthority import CertificateAuthority
from src.actors.Student import Student
from src.actors.University import University
from src.actors.Blockchain import Blockchain

from datetime import datetime, timedelta
import time

# --- SET-UP DEGLI ATTORI DEL SISTEMA ---

Per prima cosa il main provvede ad istanziare:
- uno studente;
- un’università che farà da università casa dello studente;
- un’altra università che farà da università ospitante;
- una istanza della classe CertificateAuthority che "emula" il comportamento di una CA;
- una istanza della classe Blockchain che "emula" il comportamento di uno smart contract per la memorizzazione della Certificate Revocation List sulla blockchain.

Inoltre vengono aggiunte le informazioni dello studente alla lista delle informazioni sugli studenti dell'università casa, in modo che questa possa, quando sarà richiesto, generare il certificato con le informazioni da lei autenticate.

In [2]:
# Creazione struttura contenente le informazioni dello studente
student_info = StudentInfo(
    matricola_casa="0123456789",
    matricola_ospitante="0123456789",
    nome="Mario",
    cognome="Rossi",
    email_casa="mario.rossi@studenti.casa.it",
    email_ospitante="mario.rossi@etudiant.maison.fr",
    data_di_nascita="01/01/2000",
    codice_corso_di_laurea="LM-32",
    nome_corso_di_laurea="Magistrale in Ingegneria Informatica",
    cfu_totali_conseguiti=100,
    media_voti=28.0
)

# Creazione dello studente
student = Student(
    party_id="01",
    student_info=student_info
)

# Creazione Università di origine
university_of_origin = University(
    party_id="02",
    nome="Università degli Studi di Salerno",
    nazione="Italia",
    codice_universita="IT-SA01",
    email_contatto="relazioni.internazionali@unisa.it"
)

# Aggiunta dello studente all'Università di origine
university_of_origin.add_student_info("01", student_info)

# Creazione Università ospitante
host_university = University(
    party_id="03",
    nome="Université de Rene",
    nazione="Francia",
    codice_universita="FR-RE01",
    email_contatto="relations.internationales@unirene.fr"
)

# Creazione CA
certificate_authority = CertificateAuthority("ca_01")

# Creazione della blockchain
blockchain = Blockchain()

# Descrizione generale della simulazione
La simulazione è volta a mostrare il funzionamento di tutte le fasi descritte nel Work package 3.

- Fase A1, uno studente inizia una comunicazione con la sua università d'origine;
- Fase B, lo studente richiede alla sua università di origine le proprie informazioni certificate;
- Fase A2, lo studente inizia una comunicazione con la sua università ospitante;
- Fasi C/D (1), l'università ospitante chiede delle informazioni specifiche allo studente, che le invia e riceve un ack;
- Fase E, l'università d'origine revoca il certificato dello studente;
- Fasi C/D (2), l'università ospitante chiede delle informazioni specifiche allo studente, che le invia e riceve un nack.

Questo flusso di lavoro permette di dimostrare il funzionamento delle principali funzionalità del progetto.

Nota: Dopo le fasi C/D (1) la connessione simmetrica tra studente e università ospitante è stata lasciata aperta per evitare di dover ripetere una terza volta la fase A.

## --- FASE A1: INIZIO COMUNICAZIONE STUDENTE - UNIVERSITA' ---
Lo studente e l'università di origine richiedono alla certificate authority i propri certificati, dopodiché se li scambiano e procedono all'esecuzione del protocollo di distribuzione sicura della chiave di sessione.

In [3]:
print("== FASE A == INIZIO COMUNICAZIONE STUDENTE - UNIVERSITA' DI ORIGINE ==\n")

start_time = time.perf_counter()
# Set-up informazioni per la comunicazione asimmetrica
student.set_up_asymmetric_communication_keys()
student.ask_for_certificate_of_identity(certificate_authority)

university_of_origin.set_up_asymmetric_communication_keys()
university_of_origin.ask_for_certificate_of_identity(certificate_authority)

# Scambio certificati di identità,
# lo studente invia il suo certificato firmato dalla CA all’università di origine
print("=== MESSAGGIO 1 ===")
print("Mittente     :   Studente")
print("Destinatario :   Università di origine")
print("Descrizione  :   Certificato firmato dalla CA")
print("Contenuto    :   C_Stu = E(PR_{auth}, [T_1 || ID_Stu || PU_Stu])\n")
student_certificate = student.send_certificate_of_identity()
university_of_origin.receive_certificate_of_identity(student_certificate)

# L’università di origine invia il suo certificato firmato dalla CA allo studente
print("=== MESSAGGIO 2 ===")
print("Mittente     :   Università di origine")
print("Destinatario :   Studente")
print("Descrizione  :   Certificato firmato dalla CA")
print("Contenuto    :   C_U = E(PR_{auth}, [T_2 || ID_U || PU_U])\n")
university_certificate = university_of_origin.send_certificate_of_identity()
student.receive_certificate_of_identity(university_certificate)

# Protocollo di distribuzione sicura della chiave, inizio challenge,
# l'università invia un nonce1 con la chiave pubblica dello studente
# per sfidarlo a dimostrare di possedere la chiave privata corrispondente
print("=== MESSAGGIO 3 ===")
print("Mittente     :   Università di origine")
print("Destinatario :   Studente")
print("Descrizione  :   Inizio della challenge - mutual authentication protocol")
print("Contenuto    :   E(PU_Stu, [ID_U || Nonce_1])\n")
first_message = university_of_origin.secure_key_distribution_protocol_send_first_message()

# Lo studente invia il nonce1 per dimostrare all'università di essere stato
# in grado di decrittare il messaggio di sfida e ripropone la stessa
# challenge all'università con il nonce2
print("=== MESSAGGIO 4 ===")
print("Mittente     :   Studente")
print("Destinatario :   Università di origine")
print("Descrizione  :   Risposta alla challenge e invio sfida di autenticazione all'università")
print("Contenuto    :   E(PU_U, [Nonce_1 || Nonce_2])\n")
second_message = student.secure_key_distribution_protocol_receive_first_message_and_send_second_message(first_message)

# L'università' invia il nonce2 per dimostrare allo studente di essere stata
# in grado di decrittare il messaggio di sfida
print("=== MESSAGGIO 5 ===")
print("Mittente     :   Università di origine")
print("Destinatario :   Studente")
print("Descrizione  :   Conclusione autenticazione reciproca")
print("Contenuto    :   E(PU_Stu, Nonce_2)\n")
third_message = university_of_origin.secure_key_distribution_protocol_receive_second_message_and_send_third_message(second_message)
student.secure_key_distribution_protocol_receive_third_message(third_message)
#fine challenge

# scambio della chiave di sessione, inizio comunicazione simmetrica
print("=== MESSAGGIO 6 ===")
print("Mittente     :   Università di origine")
print("Destinatario :   Studente")
print("Descrizione  :   Distribuzione chiave simmetrica")
print("Contenuto    :   E(PU_Stu, E(PR_U, K_S))\n\n")
university_of_origin.set_up_symmetric_communication()
session_info_encrypted = university_of_origin.send_information_symmetric_communication()

session_info_decrypted = student.decrypt_and_verify_message_asymmetric_encryption(session_info_encrypted)
student.set_up_symmetric_communication_from_info_received(session_info_decrypted)

end_time = time.perf_counter()
latency_ms = (end_time - start_time) * 1000  # tempo in millisecondi
print("Tempo per la presentazione tra le parti: ", latency_ms)

== FASE A == INIZIO COMUNICAZIONE STUDENTE - UNIVERSITA' DI ORIGINE ==

=== MESSAGGIO 1 ===
Mittente     :   Studente
Destinatario :   Università di origine
Descrizione  :   Certificato firmato dalla CA
Contenuto    :   C_Stu = E(PR_{auth}, [T_1 || ID_Stu || PU_Stu])

NON è passato più di un mese, certificato accettato.
=== MESSAGGIO 2 ===
Mittente     :   Università di origine
Destinatario :   Studente
Descrizione  :   Certificato firmato dalla CA
Contenuto    :   C_U = E(PR_{auth}, [T_2 || ID_U || PU_U])

NON è passato più di un mese, certificato accettato.
=== MESSAGGIO 3 ===
Mittente     :   Università di origine
Destinatario :   Studente
Descrizione  :   Inizio della challenge - mutual authentication protocol
Contenuto    :   E(PU_Stu, [ID_U || Nonce_1])

=== MESSAGGIO 4 ===
Mittente     :   Studente
Destinatario :   Università di origine
Descrizione  :   Risposta alla challenge e invio sfida di autenticazione all'università
Contenuto    :   E(PU_U, [Nonce_1 || Nonce_2])

=== MESS

## --- FASE B: RICHIESTA CERTIFICATO ALL'UNIVERSITA' ---
Lo studente richiede all'università d'origine il proprio certificato accademico autenticato

In [4]:
print("== FASE B == RICHIESTA CERTIFICATO ALL'UNIVERSITA' ==\n")

print("=== MESSAGGIO 1 ===")
print("Mittente     :   Studente")
print("Destinatario :   Università di origine")
print("Descrizione  :   Richiesta credenziali\n")
student_certificate_request = student.ask_for_student_info_certificate()

# L'università invia allo studente le credenziali con il Merkle Tree che potrà
# essere usato per verificare l'autenticità dei documenti selezionati e inviati
# a terze perti.
# Questo permette di avere maggiore privacy in quanto lo studente non è costretto
# a comunicare tutte le informazioni presenti nel Merkle Tree ma solo unicamente
# quelle necessarie.
print("=== MESSAGGIO 2 ===")
print("Mittente     :   Università di origine")
print("Destinatario :   Studente")
print("Descrizione  :   Invio credenziali con Merkle Tree per verificarne l'autenticità")
print("Contenuto    :   E(K_S, MerkleTree||E(K_U, RadiceMerkleTree))\n\n")
encrypted_info = university_of_origin.receive_student_info_certificate_request(student_certificate_request)
student.receive_student_info_certificate(encrypted_info)

# Termine comunicazione simmetrica
encrypted_end_communication_message = university_of_origin.end_symmetric_communication()
encrypted_end_communication_message = student.end_symmetric_communication()

== FASE B == RICHIESTA CERTIFICATO ALL'UNIVERSITA' ==

=== MESSAGGIO 1 ===
Mittente     :   Studente
Destinatario :   Università di origine
Descrizione  :   Richiesta credenziali

=== MESSAGGIO 2 ===
Mittente     :   Università di origine
Destinatario :   Studente
Descrizione  :   Invio credenziali con Merkle Tree per verificarne l'autenticità
Contenuto    :   E(K_S, MerkleTree||E(K_U, RadiceMerkleTree))




## --- FASE A2: INIZIO COMUNICAZIONE STUDENTE - UNIVERSITA' OSPITANTE ---
Al fine di simulare lo scambio tra studente e università ospitante è necessario ripetere la fase A, questa volta tra questi due enti.

In [5]:
print("== FASE A == INIZIO COMUNICAZIONE STUDENTE - UNIVERSITA' OSPITANTE ==\n")

start_time = time.perf_counter()
# Set-up informazioni per la comunicazione asimmetrica
host_university.set_up_asymmetric_communication_keys()
host_university.ask_for_certificate_of_identity(certificate_authority)

# Scambio certificati di identità,
# lo studente invia il suo certificato firmato dalla CA all’università di origine
print("=== MESSAGGIO 1 ===")
print("Mittente     :   Studente")
print("Destinatario :   Università ospitante")
print("Descrizione  :   Certificato firmato dalla CA")
print("Contenuto    :   C_Stu = E(PR_{auth}, [T_1 || ID_Stu || PU_Stu])\n")
student_certificate = student.send_certificate_of_identity()
host_university.receive_certificate_of_identity(student_certificate)

# L’università di origine invia il suo certificato firmato dalla CA allo studente
print("=== MESSAGGIO 2 ===")
print("Mittente     :   Università ospitante")
print("Destinatario :   Studente")
print("Descrizione  :   Certificato firmato dalla CA")
print("Contenuto    :   C_U = E(PR_{auth}, [T_2 || ID_U || PU_U])\n")
university_certificate = host_university.send_certificate_of_identity()
student.receive_certificate_of_identity(university_certificate)

# Protocollo di distribuzione sicura della chiave, inizio challenge,
# l'università invia un nonce1 con la chiave pubblica dello studente
# per sfidarlo a dimostrare di possedere la chiave privata corrispondente
print("=== MESSAGGIO 3 ===")
print("Mittente     :   Università ospitante")
print("Destinatario :   Studente")
print("Descrizione  :   Inizio della challenge - mutual authentication protocol")
print("Contenuto    :   E(PU_Stu, [ID_U || Nonce_1])\n")
first_message = host_university.secure_key_distribution_protocol_send_first_message()

# Lo studente invia il nonce1 per dimostrare all'università di essere stato
# in grado di decrittare il messaggio di sfida e ripropone la stessa
# challenge all'università con il nonce2
print("=== MESSAGGIO 4 ===")
print("Mittente     :   Studente")
print("Destinatario :   Università ospitante")
print("Descrizione  :   Risposta alla challenge e invio sfida di autenticazione all'università")
print("Contenuto    :   E(PU_U, [Nonce_1 || Nonce_2])\n")
second_message = student.secure_key_distribution_protocol_receive_first_message_and_send_second_message(first_message)

# L'università' invia il nonce2 per dimostrare allo studente di essere stata
# in grado di decrittare il messaggio di sfida
print("=== MESSAGGIO 5 ===")
print("Mittente     :   Università ospitante")
print("Destinatario :   Studente")
print("Descrizione  :   Conclusione autenticazione reciproca")
print("Contenuto    :   E(PU_Stu, Nonce_2)\n")
third_message = host_university.secure_key_distribution_protocol_receive_second_message_and_send_third_message(second_message)
student.secure_key_distribution_protocol_receive_third_message(third_message)
#fine challenge

# scambio della chiave di sessione, inizio comunicazione simmetrica
print("=== MESSAGGIO 6 ===")
print("Mittente     :   Università ospitante")
print("Destinatario :   Studente")
print("Descrizione  :   Distribuzione chiave simmetrica")
print("Contenuto    :   E(PU_Stu, E(PR_U, K_S))\n\n")
host_university.set_up_symmetric_communication()
session_info_encrypted = host_university.send_information_symmetric_communication()

session_info_decrypted = student.decrypt_and_verify_message_asymmetric_encryption(session_info_encrypted)
student.set_up_symmetric_communication_from_info_received(session_info_decrypted)

end_time = time.perf_counter()
latency_ms = (end_time - start_time) * 1000  # tempo in millisecondi
print("Tempo per la presentazione tra le parti: ", latency_ms)

== FASE A == INIZIO COMUNICAZIONE STUDENTE - UNIVERSITA' OSPITANTE ==

=== MESSAGGIO 1 ===
Mittente     :   Studente
Destinatario :   Università ospitante
Descrizione  :   Certificato firmato dalla CA
Contenuto    :   C_Stu = E(PR_{auth}, [T_1 || ID_Stu || PU_Stu])

NON è passato più di un mese, certificato accettato.
=== MESSAGGIO 2 ===
Mittente     :   Università ospitante
Destinatario :   Studente
Descrizione  :   Certificato firmato dalla CA
Contenuto    :   C_U = E(PR_{auth}, [T_2 || ID_U || PU_U])

NON è passato più di un mese, certificato accettato.
=== MESSAGGIO 3 ===
Mittente     :   Università ospitante
Destinatario :   Studente
Descrizione  :   Inizio della challenge - mutual authentication protocol
Contenuto    :   E(PU_Stu, [ID_U || Nonce_1])

=== MESSAGGIO 4 ===
Mittente     :   Studente
Destinatario :   Università ospitante
Descrizione  :   Risposta alla challenge e invio sfida di autenticazione all'università
Contenuto    :   E(PU_U, [Nonce_1 || Nonce_2])

=== MESSAGGIO

## --- FASE C: INVIO CERTIFICATO ALL'UNIVERSITA' e FASE D VERIFICA CERTIFICATO ---
L'università ospitante richiede una certa informazione allo studente, quest'ultimo la fornisce insieme alla merkle proof necessaria a verificare l'autenticità dei dati. Infine l'università ospitante fornisce un ACK per confermare che i dati erano corretti.
La comunicazione simmetrica tra le due parti non viene chiusa in quanto si intende riprovare questa fase dopo aver revocato il certificato e si vuole evitare una terza esecuzione della fase per brevità.

In [6]:
print("== FASE C == INVIO CERTIFICATO ALL'UNIVERSITA' ==\n")

# L'università fa una richiesta di informazioni secifiche allo studente
print("=== MESSAGGIO 1 ===")
print("Mittente     :   Università ospitante")
print("Destinatario :   Studente")
print("Descrizione  :   Richiesta informazioni specifiche")
print("Contenuto    :   E(K_S, Richiesta mail_casa)\n")
encrypted_request = host_university.request_info("email_casa")

# Lo studente invia le informazioni alleganto il Merkle Tree e tutti
# i gli Hash dei nodi aggiuntivi necessari per il calcolo dell'autenticità
print("=== MESSAGGIO 2 ===")
print("Mittente     :   Studente")
print("Destinatario :   Università ospitante")
print("Descrizione  :   Informazioni specifiche richiesta con Merkle Tree per verificarte l'auteticità")
print("Contenuto    :   E(K_S, foglieRichiesteDelMerkleTree||nodiAggiuntiviDelMerkleTreePerIlCalcoloDellaRadice||E(K_U, RadiceMerkleTree))\n\n")
encrypted_info_student = student.receive_request_info_and_send_info(encrypted_request)

== FASE C == INVIO CERTIFICATO ALL'UNIVERSITA' ==

=== MESSAGGIO 1 ===
Mittente     :   Università ospitante
Destinatario :   Studente
Descrizione  :   Richiesta informazioni specifiche
Contenuto    :   E(K_S, Richiesta mail_casa)

=== MESSAGGIO 2 ===
Mittente     :   Studente
Destinatario :   Università ospitante
Descrizione  :   Informazioni specifiche richiesta con Merkle Tree per verificarte l'auteticità
Contenuto    :   E(K_S, foglieRichiesteDelMerkleTree||nodiAggiuntiviDelMerkleTreePerIlCalcoloDellaRadice||E(K_U, RadiceMerkleTree))




In [7]:
print("== FASE D == VERIFICA CERTIFICATO ==\n")

print("=== MESSAGGIO 1 ===")
print("Mittente     :   Università ospitante")
print("Destinatario :   Studente")
print("Descrizione  :   Notifica di ricezione di certificato corretto o no")
print("Contenuto    :   E(K_S, ack/nack)\n")
# L'università verifica le informazioni e notifica lo studente con un riscontro positivo o negativo
# l'università ospitante ha bisogno di conoscere l'identità dell'università casa, per semplicità si passa la chiave pubblica come argomento
ack_nack = host_university.receive_info_requested(encrypted_info_student, university_of_origin.asymmetric_encryption_information.public_key, blockchain)

student.receive_feedback_on_info_student(ack_nack)

# manteniamo aperta la comunicazione in modo da non doverla riaprire per verificare se la revoca funziona
#host_university.end_symmetric_communication()
#student.end_symmetric_communication()

== FASE D == VERIFICA CERTIFICATO ==

=== MESSAGGIO 1 ===
Mittente     :   Università ospitante
Destinatario :   Studente
Descrizione  :   Notifica di ricezione di certificato corretto o no
Contenuto    :   E(K_S, ack/nack)

ACK
Il certificato delle informazioni sullo studente è stato 
ACCETTATO


## --- FASE E: REVOCA CERTIFICATO ---
L'università di origine esegue la revoca del certificato rilasciato allo studente aggiungendo il codice del certificato alla Certificate Revocation List ospitata dalla blockchain.

In [8]:
print("== FASE E == REVOCA CERTIFICATO ==\n")

print("=== MESSAGGIO 1 ===")
print("Mittente     :   Università ospitante")
print("Destinatario :   Studente")
print("Descrizione  :   Aggiunta alla CRL sulla blockchain del codice del certificato da revocare")
university_of_origin.revoke_certificate("01", blockchain)

== FASE E == REVOCA CERTIFICATO ==

=== MESSAGGIO 1 ===
Mittente     :   Università ospitante
Destinatario :   Studente
Descrizione  :   Aggiunta alla CRL sulla blockchain del codice del certificato da revocare
certificato di 01 revocato


## --- FASE C: INVIO CERTIFICATO ALL'UNIVERSITA' e FASE D VERIFICA CERTIFICATO ---
L'università ospitante richiede una certa informazione allo studente, quest'ultimo la fornisce insieme alla merkle proof necessaria a verificare l'autenticità dei dati. Infine l'università ospitante fornisce un NACK in quanto il certificato risulta revocato.

In [9]:
print("== FASE C == INVIO CERTIFICATO ALL'UNIVERSITA' ==\n")

# L'università fa una richiesta di informazioni specifiche allo studente
print("=== MESSAGGIO 1 ===")
print("Mittente     :   Università ospitante")
print("Destinatario :   Studente")
print("Descrizione  :   Richiesta informazioni specifiche")
print("Contenuto    :   E(K_S, Richiesta mail_casa)\n")
encrypted_request = host_university.request_info("email_casa")

# Lo studente invia le informazioni alleganto il Merkle Tree e tutti
# i gli Hash dei nodi aggiuntivi necessari per il calcolo dell'autenticità
print("=== MESSAGGIO 2 ===")
print("Mittente     :   Studente")
print("Destinatario :   Università ospitante")
print("Descrizione  :   Informazioni specifiche richiesta con Merkle Tree per verificarte l'auteticità")
print("Contenuto    :   E(K_S, foglieRichiesteDelMerkleTree||nodiAggiuntiviDelMerkleTreePerIlCalcoloDellaRadice||E(K_U, RadiceMerkleTree))\n\n")
encrypted_info_student = student.receive_request_info_and_send_info(encrypted_request)

== FASE C == INVIO CERTIFICATO ALL'UNIVERSITA' ==

=== MESSAGGIO 1 ===
Mittente     :   Università ospitante
Destinatario :   Studente
Descrizione  :   Richiesta informazioni specifiche
Contenuto    :   E(K_S, Richiesta mail_casa)

=== MESSAGGIO 2 ===
Mittente     :   Studente
Destinatario :   Università ospitante
Descrizione  :   Informazioni specifiche richiesta con Merkle Tree per verificarte l'auteticità
Contenuto    :   E(K_S, foglieRichiesteDelMerkleTree||nodiAggiuntiviDelMerkleTreePerIlCalcoloDellaRadice||E(K_U, RadiceMerkleTree))




In [10]:
print("== FASE D == VERIFICA CERTIFICATO ==\n")

print("=== MESSAGGIO 1 ===")
print("Mittente     :   Università ospitante")
print("Destinatario :   Studente")
print("Descrizione  :   Notifica di ricezione di certificato corretto o no")
print("Contenuto    :   E(K_S, ack/nack)\n")
# L'università verifica le informazioni e notifica lo studente con un riscontro positivo o negativo
# l'università ospitante ha bisogno di conoscere l'identità dell'università casa, per semplicità si passa la chiave pubblica come argomento
ack_nack = host_university.receive_info_requested(encrypted_info_student, university_of_origin.asymmetric_encryption_information.public_key, blockchain)

student.receive_feedback_on_info_student(ack_nack)

# Termine della comunicazione cifrata
encrypted_end_communication_message = host_university.end_symmetric_communication()
encrypted_end_communication_message = student.end_symmetric_communication()

== FASE D == VERIFICA CERTIFICATO ==

=== MESSAGGIO 1 ===
Mittente     :   Università ospitante
Destinatario :   Studente
Descrizione  :   Notifica di ricezione di certificato corretto o no
Contenuto    :   E(K_S, ack/nack)

NACK
Il certificato delle informazioni sullo studente è stato 
RIFIUTATO
